In [1]:
import os
import scvi
import torch
import pandas as pd
import scanpy as sc
import anndata as ad

# conda activate anndata

torch.set_num_threads(20)
# OMP_NUM_THREADS=20

# pd.set_option("display.max_columns", None)

os.chdir("/mnt/lareaulab/reliscu/projects/Chronocell/analyses/janssens_2025_preprint")

/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here I run scVI to get a latent representation of the single-cell data for visualization purposes

In [ ]:
adata = ad.read_h5ad("/mnt/lareaulab/reliscu/projects/Chronocell/data/janssens_2025_preprint/janssens_2025_CellRanger_counts.h5ad")

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.raw = adata  # Keep full dimension safe

In [ ]:
# adata.obs['Experimental_condition'] = adata.obs['Condition'].str.split(r" [0-9]").str[0]

In [ ]:
# Subset data to highly variable features

sc.pp.highly_variable_genes(
  adata,
  n_top_genes = 2000,
  subset = True,
  layer = "counts",
  flavor = "seurat_v3", 
  batch_key="Condition"
)

In [ ]:
# scVI model

# Note: this takes about 30+ minutes
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="Condition")
model = scvi.model.SCVI(adata)
model.train()

In [ ]:
# Extract low dim rep.
latent = model.get_latent_representation()
adata.obsm["scVI"] = latent

# Save
model.save("data/janssens_2025_CellRanger_model_Condition")
adata.write("data/janssens_2025_CellRanger_model_Condition/janssens_2025_CellRanger_counts_scVI_Condition.h5ad")